In [46]:
import sys
sys.path.append("C:/Users/Vincent/.snap/snap-python")

from osgeo import gdal,ogr, osr
gdal.UseExceptions()
ogr.UseExceptions()
osr.UseExceptions()

import snappy
from snappy import ProductIO
from snappy import HashMap
from snappy import GPF
from snappy import jpy

import os
import os.path
from collections import OrderedDict
from datetime import datetime
import datetime
from datetime import timedelta, date
import time

import numpy as np
import glob
from itertools import cycle
import math
import shutil
import json
import geojson
import random

import array
import numpy as np

from keras.models import Model
from keras.layers.core import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input
from sklearn import metrics as me
from scipy import stats

outFormat = 'ESRI Shapefile'

GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()
HashMap = snappy.jpy.get_type('java.util.HashMap')

System = jpy.get_type('java.lang.System')
System.gc()

PrintWriterProgressMonitor = jpy.get_type('com.bc.ceres.core.PrintWriterProgressMonitor')
FileOutputStream = jpy.get_type('java.io.FileOutputStream')

processTimeInFunction = None	# Used to measure the time needed to processing the data in a function
writeTimeInFunction = None		# Used to measure the time needed to write the data in a function

projectName = "Test_TM"
home = "H:/TM/V2/Neural_Terra-i/result"

folderPathsSubsets = []
folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/1")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/2")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/3")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/4")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/5")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/6")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/7")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/8")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/9")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Bresil/10")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Highlands/1")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Highlands/2")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Highlands/3")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Highlands/4")
#folderPathsSubsets.append("H:/TM/V2/data/subset/Highlands/5")

folderPathsSubsetsResults = []
folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/1")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/2")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/3")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/4")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/5")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/6")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/7")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/8")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/9")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Bresil/10")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Highlands/1")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Highlands/2")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Highlands/3")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Highlands/4")
#folderPathsSubsetsResults.append("H:/TM/V2/data/subset_Result/Highlands/5")

In [18]:
def CreateFolder(LocalityPath):
    if not os.path.exists(LocalityPath):
        os.makedirs(LocalityPath)

# Step 0
def GetProductList():
    productList = []
    unzipAllFileArchives(folderPath)
    for folder in os.listdir(folderPath):
        product = ProductIO.readProduct(os.path.join(folderPath,folder, "manifest.safe"))
        productList.append(product)
    return productList

def GetProductListDim(paths):
    
    products = []
    
    for path in paths :
        
        lbs_input_path = os.path.join(path,'*.dim')
        files = glob.glob(lbs_input_path)

        for f in files:
            product = ProductIO.readProduct(f)
            products.append(product)

    return products

# Step 1
def GetApplyOrbit(products, write_product):

    global writeTimeInFunction
    global processTimeInFunction

    startTimeProcess = time.time()

    orbit_list = []
    orbit_folder = os.path.join(home, projectName, 'Results', '1.ApplyOrbit', '')
    CreateFolder(orbit_folder)
    for product in products:
        orbit_name = product.getName() + '.dim'
        orbit_path = os.path.join(orbit_folder, orbit_name)

        if not os.path.exists(orbit_path):
            #print('Processing: ' + orbit_name)
            orbit_hashmap = HashMap()
            orbit_hashmap.put('continueOnFail', True)
            orbit_hashmap.put('Apply-Orbit-File', True)
            orbit = GPF.createProduct("Apply-Orbit-File", orbit_hashmap, product)

            orbit_list.append(orbit)

            #fileOutputStream = FileOutputStream(orbit_folder + "\\1.progress.txt")
            #printWriterProgressMonitor = PrintWriterProgressMonitor(fileOutputStream)

            PWPM = jpy.get_type('com.bc.ceres.core.PrintWriterProgressMonitor')
            JavaSystem = jpy.get_type('java.lang.System')
            monitor = PWPM(JavaSystem.out)

            if write_product:
                startTimeWrite = time.time()
                ProductIO.writeProduct(orbit, orbit_path, 'BEAM-DIMAP', monitor)
                writeTimeInFunction = time.time() - startTimeWrite
            orbit.dispose()
            product.dispose()
            del orbit
            del product
            del orbit_hashmap
    
    del products
    products = None
    print('APPLY ORBIT..................................DONE\n')

    processTimeInFunction = time.time() - startTimeProcess

    return(orbit_list)

In [14]:
subsets = GetProductListDim(folderPathsSubsets)
subsetsResults = GetProductListDim(folderPathsSubsetsResults)

print(len(subsets))
print(len(subsetsResults))

100
100


In [23]:
GetApplyOrbit(subsets, True)
orbitPaths = []
orbitPaths.append(os.path.join(home, projectName, 'Results', '1.ApplyOrbit', ''))
orbits = GetProductListDim(orbitPaths)

del subsets

print(len(subsets))
print(len(orbits))

APPLY ORBIT..................................DONE

org.esa.snap.core.datamodel.Product[name=S1A_IW_GRDH_1SDV_20210826T101413_20210826T101438_039399_04A776_7B55_0]


NameError: name 'subsets' is not defined

In [47]:
bandNames = orbits[0].getBandNames()

for name in bandNames:
    print(name)
    
tieNames = orbits[0].getTiePointGridNames()

for tieName in tieNames:
    print(tieName)
    
temp =  orbits[0].getBands()

for t in temp:
    print(t)

w = temp[0].getRasterWidth()
h = temp[0].getRasterHeight()

pixels = array.array('i', (0 for i in range(0, w * h)))

temp[0].readPixels(0, 0, w, h, pixels)

tempImage = np.copy(pixels) # deep copy

pixels[0] = -1
print(pixels[0])
print(len(tempImage))
print(tempImage[0])

Amplitude_VH
Intensity_VH
Amplitude_VV
Intensity_VV
latitude
longitude
incident_angle
elevation_angle
slant_range_time
org.esa.snap.core.datamodel.Band[Amplitude_VH,int16,763,1019,-1,0.0,0.0,0.0]
org.esa.snap.core.datamodel.VirtualBand[Intensity_VH,float32,763,1019]
org.esa.snap.core.datamodel.Band[Amplitude_VV,int16,763,1019,-1,0.0,0.0,0.0]
org.esa.snap.core.datamodel.VirtualBand[Intensity_VV,float32,763,1019]
-1
777497
99


In [52]:
print(orbits[-1])

org.esa.snap.core.datamodel.Product[name=S1A_IW_GRDH_1SDV_20210826T101413_20210826T101438_039399_04A776_7B55_99]


In [56]:
X = []

for orbit in orbits[:1]:
    
    channels = []
    
    w = orbit.getBands()[0].getRasterWidth()
    h = orbit.getBands()[0].getRasterHeight()
    
    for band in orbit.getBands():
     
        channels.append(array.array('i', (0 for i in range(0, w * h)))) 
        
        band.readPixels(0, 0, w, h, channels[-1])
        
    for tiePointGrid in orbit.getTiePointGrids():
        
        channels.append(array.array('i', (0 for i in range(0, w * h))))
        
        tiePointGrid.readPixels(0, 0, w, h, channels[-1])
       
    for i in len(channels[i]):
        
        channels[i] = np.array_split(channels[i], h)

777497
99
777497
9801
777497
221
777497
48841
777497
0
777497
0
777497
0
777497
0
777497
0


In [ ]:
l0 = Input(shape=(height, width, 9), name='l0')

l1 = Conv2D(9, (5, 5), padding='same', activation='relu', name='l1')(l0)
l1_mp = MaxPooling2D(pool_size=(2, 2), name='l1_mp')(l1)

l2 = Conv2D(9, (5, 5), padding='same', activation='relu', name='l2')(l1_mp)
l2_mp = MaxPooling2D(pool_size=(2, 2), name='l2_mp')(l2)

l3 = Conv2D(16, (3, 3), padding='same', activation='relu', name='l3')(l2_mp)
l3_mp = MaxPooling2D(pool_size=(2, 2), name='l3_mp')(l3)

flat = Flatten(name='flat')(l3_mp)

l4 = Dense(25, activation='relu', name='l4')(flat)

l5 = Dense(n_classes, activation='softmax', name='l5')(l4)

model = Model(inputs=l0, outputs=l5)
model.summary()

In [68]:
productTest = ProductIO.readProduct("H:/TM/V2/data/subset_Result/Highlands/5/S1A_IW_GRDH_1SDV_20211223T223727_20211223T223755_041142_04E37D_8686_0.dim")

channels = []

w = 0
h = 0

for band in productTest.getBands():
    
    print(band)
   
    w = band.getRasterWidth()
    h = band.getRasterHeight()
    
    print(w)
    print(h)

    channels.append(array.array('i', (0 for i in range(0, w * h))))

    band.readPixels(0, 0, w, h, channels[-1])
    
valueMin = 9999999
valueMax = -9999999
    
for channel in channels:
    
    for i in range(0, len(channel)):
        
        if channel[i] > valueMax:
            
            valueMax = channel[i]
        
        if channel[i] < valueMin:
            valueMin = channel[i]
            
print(valueMin)
print(valueMax)

print(len(channels[0]))

channels[0] = np.array_split(channels[0], h)
print(len(channels[0]))
print(len(channels[0][0]))

org.esa.snap.core.datamodel.Band[Sigma0_VH,float32,1236,957,-1,0.0,0.0,0.0]
1236
957
org.esa.snap.core.datamodel.Band[Sigma0_VV,float32,1236,957,-1,0.0,0.0,0.0]
1236
957
0
3
1182852
957
1236
